Loopy graphs
===

The schedules demo already briefly mentioned loopy graphs. An example graph is given below.

```
    (driver)
   -->[A]---
   |       |
   |      [+]<-[N]
   |       |
   ---[B]<--
  (inhibitor)
```

A graph with loops must have its cycles 'opened' before a schedule can be generated. We need to set a 'breaker' message at certain interfaces to provide the autoscheduler with a place to start. Since we have no knowledge about the graph beyond our breaker message, we start with a vague message that represents out ignorance. The `vague()` function can be used to produce vague (broad, quite uninformative) distributions.

Calculating a message within the loop
---

Suppose we want to calculate the outgoing message on the driver node, at `driver.out`. Let's first define the graph. 

In [1]:
using ForneyLab

# Define the graph
driver      = FixedGainNode([1.1], name="driver")
inhibitor   = FixedGainNode([0.1], name="inhibitor")
noise       = TerminalNode(GaussianDistribution(m=0.0, V=0.1), name="noise")
add         = AdditionNode(name="adder")
Edge(add.out, inhibitor.in1)
Edge(inhibitor.out, driver.in1)
Edge(driver.out, add.in1)
Edge(noise.out, add.in2);

#SumProduct.generateSchedule(driver.out) # Uncomment to produce error

Edge from TerminalNode noise:1 to AdditionNode adder:2.


The automated schedule generator for defining algorithms is dependent on present messages in the graph. Defining an algorithm in a loopy graph without predefining any breaker messages will throw an error like: 

>Loop detected around Interface 2 (out) of FixedGainNode driver.
 Consider setting an initial message somewhere in this loop.

Let's follow this advise and set some initial messages before generating a schedule:

Because we want to calculate the message at the outgoing interface, we need to break the loop there as well. Let's set the breaker message and generate a schedule.

In [2]:
# Set a relatively uninformative breaker message
breaker_message = Message(vague(GaussianDistribution))
setMessage!(driver.out, breaker_message);

We can generate a sumproduct schedule that calculates a new value for `driver.out`:

In [3]:
schedule = SumProduct.generateSchedule(driver.out)
show(schedule)

Message passing schedule (entry: node [interface], rule)
------------------------------------------------------
1: TerminalNode noise [1:out], sumProduct! 
2: AdditionNode adder [3:out], sumProduct! 
3: FixedGainNode inhibitor [2:out], sumProduct! 
4: FixedGainNode driver [2:out], sumProduct! 


And there it is, we have our schedule. Next we can use this schedule to perform our calculations. We construct a custom `execute()` function that evaluates the schedule over and over and construct an algorithm with it.

In [4]:
# Construct custom update function that executes the schedule ten times
function exec(fields)
    for itr = 1:10
        execute(fields[:schedule])
    end
end

# Construct loopy sumproduct algorithm that executes exec with the proper schedule
algo = Algorithm(exec, {:schedule => schedule})

Algorithm with fields:
 schedule::Array{ScheduleEntry,1}

Use algorithm.fields[:field] to inspect field values.


In [5]:
# Run the algorithm for one step
step(algo)
show(driver.out.message)

Message{GaussianDistribution} with payload N(m=[0.00], V=[[1.22e-03]])



Calculating a message outside the loop
---

Now suppose we want to calculate the message from the addition node towards the noise node, at `add.in2`. In this case, calculating the message at `add.in1` requires the message at `add.out` and vice versa, because of the loop. This indicates that we need to set breaker messages in both directions.

But first we need to clear all messages in the graph so we can create a new algorithm.

In [6]:
clearMessages!();

Now we have a clean slate again, so we can set the breaker messages and generate the schedule.

In [7]:
setMessage!(add.in1, Message(vague(GaussianDistribution)))
setMessage!(add.out, Message(vague(GaussianDistribution)))
schedule2 = SumProduct.generateSchedule(add.in2) # Generate the schedule based on the set breaker message
show(schedule2)

Message passing schedule (entry: node [interface], rule)
------------------------------------------------------
1: FixedGainNode inhibitor [2:out], sumProduct! 
2: FixedGainNode driver [2:out], sumProduct! 
3: FixedGainNode driver [1:in1], sumProduct! 
4: FixedGainNode inhibitor [1:in1], sumProduct! 
5: AdditionNode adder [2:in2], sumProduct! 


Again we can execute the schedule as many times as we desire.

In [8]:
algo2 = Algorithm(exec, {:schedule => schedule2})
step(algo2)
show(add.in2.message)

Message{GaussianDistribution} with payload N(m=[0.00], V=[[8.27e+13]])

